In [ ]:
# We have switch to using xarray, this is why we have a second notebook for data exploration.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import os
import pathlib

import pandas as pd
import altair as alt
import numpy as np
import xarray as xr

import plotly.express as px

alt.data_transformers.disable_max_rows()

from psp.data.uk_pv import C, trim_pv, filter_rows, get_max_power_for_time_of_day


def _(df, *args, **kwargs):
    print(len(df))
    display(df.head(*args, **kwargs))

In [ ]:
# It's always annoying to set the working directory: we use an environment variable defined in the Makefile.
CWD = os.environ.get("CWD")
if CWD:
    os.chdir(CWD)

In [ ]:
df = xr.open_dataset("data/5min_2.netcdf")

In [ ]:
df

In [ ]:
%%time
x = df.rename_vars({"generation_wh": "power"}).rename_dims(
    {"timestamp": "ts", "ss_id": "id"}
)
for _ in range(100):
    x.isel(ts=slice(1, 3), id=slice(1, 3))
    # x.isel(timestamp=slice(1, 3), ss_id=slice(1, 3))

In [ ]:
counts = df.groupby_bins("timestamp", bins=4 * 12).count()
counts = counts.to_dataframe().reset_index()
counts["timestamp_bins"] = counts["timestamp_bins"].apply(lambda x: x.left)

In [ ]:
data = counts.copy()
chart = (
    alt.Chart(data)
    .mark_rect()
    .encode(y=C.id + ":O", x="timestamp_bins:O", color=alt.Color(C.power + ":Q"))
)
chart

In [ ]:
# rendu ici, faire un histogramme et trouver où couper... meme si on a deja ca qqpart!
counts2 = df.count("timestamp")
counts2.to_dataframe()

In [ ]:
data = counts2.to_dataframe().reset_index()
chart = (
    alt.Chart(data)
    .mark_bar()
    .encode(y="count()", x=alt.X(C.power, bin=alt.Bin(maxbins=200)))
)
chart

In [ ]:
# ss_ids with too little data
data = counts2.to_dataframe().reset_index().copy()
_(data)
enough_data = data[C.power] > 10000
train = data[C.id] < 25000
test = data[C.id] > 25000

print((train & enough_data).sum())
print((test & enough_data).sum())
print((~enough_data).sum())
print(data[~enough_data][C.id].to_list())

In [ ]:
df
df.sel(ss_id=(df["ss_id"] % 10 == 0))

In [ ]:
df.ss_id % 10

In [ ]:
x = df.sel(ss_id=8229)

In [ ]:
x.coords.values()

In [ ]:
# coords = x.coords.values()['latitude']
coords["latitude"].values